# <span style='color:slateblue'>Trabalho #2 de Aprendizado de Máquina - UFMG - 2018/1</span>

## <span style='color:slateblue'>Eduardo Santiago Ramos</span>
## <span style='color:slateblue'>Prof. Adriano Veloso</span>

# <span style='color:crimson'>0.</span> Enunciado

### Descrição
Sabemos que erros de um modelo de previsão podem ser decompostos em dois fatores: viés
e variância. Um modelo complexo apresenta pouco viés, mas grande variância, enquanto um
modelo simples apresenta grande viés e pouca variância.
Boosting é o processo de reduzir o viés de um grande conjunto de modelos simples (i.e.,
com baixa variância). Esses modelos são chamados de modelos fracos, e são levemente correlacionados com a classificao correta. No processo de Boosting, os modelos fracos formam
um modelo mais forte de maneira iterativa. Os modelos fracos são escolhidos iterativamente,
de forma que cada modelo é escolhido levando-se em conta viéses independentes. Ou seja,
cada modelo componente realiza erros diferentes de outros modelos componentes.
### Objetivo
Nosso foco neste trabalho prático é ganhar experiência com o processo de Boosting. Você deverá implementar o processo de Boosting (visto em sala de aula) assumindo um problema de
classificação binária com atributos categóricos. Em particular, você deverá realizar os experimentos utilizando o dataset tic-tac-toe, disponível em https://archive.ics.uci.edu/ml/
datasets/Tic-Tac-Toe+Endgame. Sua avaliação deverá seguir a metodologia de validação
cruzada com 5 partições. A medida de eficácia a ser considerada é a taxa de erro simples. Voc
deverá implementar todo o processo de Boosting, e portanto não é permitido utilizar soluções
prontas (embora não seja necessário implementar funções básicas). A escolha da linguagem
de programação é uma decisão do aluno.
### Entregáveis
Você deverá entregar, via Moodle da disciplina e dentro do prazo, arquivo contendo o código
fonte do programa desenvolvido, bem como a documentação. A documentação deve conter
toda a informação referente a suas escolhas de implementação. Além disso, você deverá
fazer uma série de análises de erro, e reportar os resultados obtidos. Os gráficos a serem
apresentados devem refletir experimentos que avaliem aspectos relacionados ao processo de
Boosting, como por exemplo o número de iterações.

# <span style='color:crimson'>0.</span> Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection as skm
import sklearn.preprocessing as skp

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

# <span style='color:crimson'>1.</span> Load Dataset

Fonte: https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame

5. Number of Instances: 958 (legal tic-tac-toe endgame boards)

6. Number of Attributes: 9, each corresponding to one tic-tac-toe square

7. Attribute Information: (`x`=player x has taken, `o`=player o has taken, `b`=blank)

    1. top-left-square: {x,o,b}
    2. top-middle-square: {x,o,b}
    3. top-right-square: {x,o,b}
    4. middle-left-square: {x,o,b}
    5. middle-middle-square: {x,o,b}
    6. middle-right-square: {x,o,b}
    7. bottom-left-square: {x,o,b}
    8. bottom-middle-square: {x,o,b}
    9. bottom-right-square: {x,o,b}
    - Class: {`positive`,`negative`}

8. Missing Attribute Values: None

9. Class Distribution: About 65.3% are positive (i.e., wins for "x")

In [2]:
with open('tic-tac-toe_data','rb') as f:
    raw_data = f.readlines()

dataset = np.empty((len(raw_data), 10), dtype=str) # number of samples x (9 positions + result) of type str
for i, sample in enumerate(raw_data):
    sample = sample.decode('utf-8')                # from bytes to str
    dataset[i,:] = sample.strip('\n').split(',')   # add sample

In [3]:
dataset

array([['x', 'x', 'x', ..., 'o', 'o', 'p'],
       ['x', 'x', 'x', ..., 'x', 'o', 'p'],
       ['x', 'x', 'x', ..., 'o', 'x', 'p'],
       ...,
       ['o', 'x', 'o', ..., 'o', 'x', 'n'],
       ['o', 'x', 'o', ..., 'o', 'x', 'n'],
       ['o', 'o', 'x', ..., 'x', 'x', 'n']], dtype='<U1')

## <span style='color:orange'>1.1.</span> Verificação da presença de ruído no dataset

Verifica se os resultados presentes no dataset correspondem ao resultado real do jogo, ou se há algum "ruído", i.e., falso positivo/negativo.

In [4]:
# Verifica dataset - se ha ruido ou o resultado dos jogos e' o correto
jogo_res = np.full(958,fill_value='n')
win_possibilities = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
for ind,sample in enumerate(dataset):
    for (i,j,k) in win_possibilities:
        if sample[i]=='x' and sample[j]=='x' and sample[k]=='x':
            jogo_res[ind]='p'
all(jogo_res==dataset[:,-1])

True

### <span style='color:darkmagenta'>Conclusão</span>:
O dataset é **determinístico**, ou seja, os resultados presentes de fato correspondem aos resultados determinados pelas regras.

# <span style='color:crimson'>1.</span> Função de custo do CART e índice Gini 

Para escolher a melhor divisão para os stumps, é necessário uma função objetivo. Neste trabalho, será utilizado a função objetivo do algoritmo CART. Além disso, será utilizado como métrica o índice **Gini** (um outro critério muito utilizado é a entropia).

Dessa forma, a função de custo do CART utilizada aqui é

$$J(k,t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$$

Como necessitamos ponderar as amostras a cada novo stump, ela será adaptada para

$$J(k,t_k) = \frac{\sum w_{left}}{\sum w}G_{left} + \frac{\sum w_{right}}{\sum w}G_{right}$$

em que $w_{left}$ é o peso de cada amostra do filho à esquerda e $w_{right}$ do filho à direita. Os valores dos índices Gini também foram adaptados para utilizar os pesos de cada amostra.

In [5]:
def CART_cost(y, w, idx):
    children = (idx,~idx)                    # filhos da esquerda e da direita
    ch_C = []                                # classe estimada para cada filho
    ch_P = []                                # probabilidades para cada filho
    ch_G = []                                # indice Gini para cada filho
    w_sum = np.sum(w)                        # soma de todos os pesos
    J = 0                                    # Custo CART
    for child in children:
        # Caso algum dos filhos nao possua amostra
        if all(child==False):
            ch_C.append(None)
            ch_G.append(None)
            ch_P.append(None)
            continue
            
        w_ch = w[child]                      # pesos no nó
        y_ch = y[child]                      # classes no nó
        w_sum_ch = np.sum(w_ch)              # soma pesos do filho
        G = 1                                # indice Gini
        max_w_sum_ch_cls = 0                 # maior soma ponderada de pesos do filho
        for cls in set(y_ch):
            w_sum_ch_cls = np.sum(w_ch[y_ch==cls]) # soma ponderada da classe do filho
            G -= (w_sum_ch_cls/w_sum_ch)**2        # quadrado da soma ponderada 
            if w_sum_ch_cls > max_w_sum_ch_cls:    
                max_w_sum_ch_cls = w_sum_ch_cls
                pred_cls = cls
                
        ch_C.append(pred_cls) 
        ch_G.append(G)        
        ch_P.append(max_w_sum_ch_cls/w_sum_ch) 
        J += G*w_sum_ch/w_sum 
        
    return J, ch_C, ch_G, ch_P

# <span style='color:crimson'>2.</span> Métrica de avaliação

A métrica de avaliação é a **taxa de erro**. Há duas versões de interesse:
- taxa de erro com pesos iguais para todas as amostras
- taxa de erro ponderada - com pesos de cada amostra

In [6]:
hit_rate = lambda y_pred,y_real: float(np.sum(y_pred==y_real)/len(y_pred))
weighted_hit_rate = lambda y_pred,y_real,w: float(np.sum(w[y_pred==y_real])/np.sum(w))

# <span style='color:crimson'>3.</span> Decision Stump

A função a seguir cria um novo stump dados um conjunto de treinamento $(X,y)$ e um vetor de pesos $w$.

In [7]:
# CART para classificacao
def create_node(X, y, w, enable_print=False):
    # Para cada atributo
    J_min = np.inf
    for att in range(X.shape[-1]):
        # Para cada valor do atributo
        for val in set(X[:,att]):
            # Divide o dataset
            idx = X[:,att]==val

            # Avalia função ponderada de custo do CART
            J, children_Preds, children_Ginis, children_Probs = CART_cost(y, w, idx)

            # Se for o melhor custo, salva o nó
            if J < J_min:
                J_min = J
                node_att = att
                node_val = val
                node_ch_Preds = children_Preds
                node_ch_Probs = children_Probs
                node_ch_Ginis = children_Ginis
                node = {'Explanation': 'x[%d] == %s'%(node_att, node_val),
                        'Condition': lambda x: x[node_att]==node_val, 
                        'CART_cost': J_min,
                        'Prediction': {True: node_ch_Preds[0], False: node_ch_Preds[1]},
                        'Children_Probs': {True: node_ch_Probs[0], False: node_ch_Probs[1]},
                        'Children_Ginis': {True: node_ch_Ginis[0], False: node_ch_Ginis[1]}
                       }

    if enable_print:
        print('Classifique como "%s" se x[%d]==%s; "%s", caso contrario'%(node['Prediction'][True], 
                                                                          node_att, node_val,
                                                                          node['Prediction'][False])
             ) 
    
    return node

def print_node(node):
    for i,j in node.items():
        print(i,': ', j)

evaluate_node = lambda node,X: np.array([node['Prediction'][node['Condition'](x)] for x in X])

# <span style='color:crimson'>4.</span> *AdaBoost*

O algoritmo do *AdaBoost* utiliza uma série de preditores fracos (stumps, no nosso caso) para montar um ensemble forte, reponderando as amostras de acordo com os erros dos preditores anteriores.

Será utilizada a função anterior de criar stumps para realizar este processo.

## <span style='color:orange'>4.1.</span> Função auxiliar: retorna predições de um ensemble (do AdaBoost)

In [8]:
def evaluate_ensemble(ensemble, X):
    y_pred = np.full(X.shape[0], fill_value='n')
    for i,x in enumerate(X):
        votes = {'n':0, 'p':0}
        for alpha,stump in ensemble:
            votes[evaluate_node(stump, [x])[0]] += alpha
        if votes['p'] >= votes['n']:
            y_pred[i] = 'p'
    return y_pred

## <span style='color:orange'>4.2.</span> Função de treinamento

Parâmetros:
- Matriz de atributos $X$
- Vetor de classes $y$
- Número de preditores $n_{stumps}$

In [9]:
def fit_adaboost(X, y, n_stumps, learning_rate=1, enable_print=False, get_errors=False):
    nt = len(y_train)

    ensemble = []
    w = np.ones(nt) / nt

    # Iteracao: adicao de novos stumps
    w_error_ens = []     # taxa de erro ponderado do ensemble
    w_error_stump = [] # taxa de erro ponderado de stumps individuais
    hist_data = []
    for i in range(n_stumps):
        # Imprime
        if enable_print==True: 
            print('Stump #%d'%i, end=' - ')
            hist_data.append(go.Histogram(x=w.copy(), opacity=0.75))
            
        # Criacao de novo stump
        stump = create_node(X=X, y=y, w=w, enable_print=enable_print)

        # Avalia taxa de erro ponderada
        y_pred_stump = evaluate_node(stump, X)
        w_error = 1 - weighted_hit_rate(y_real=y, y_pred=y_pred_stump, w=w)
        if get_errors==True:
            w_error_stump.append(w_error)

        # Peso do preditor
        alpha = learning_rate*np.log((1-w_error)/w_error)

        # Adiciona o preditor 'a lista 
        ensemble.append((alpha, stump))
        
        # Avalia taxa de erro ponderada
        if get_errors==True:
            y_pred_ens = evaluate_ensemble(ensemble, X)
            w_error = 1 - weighted_hit_rate(y_real=y, y_pred=y_pred_ens, w=w)
            w_error_ens.append(w_error)

        # Atualiza pesos 
        w[y_pred_stump!=y] = w[y_pred_stump!=y] * np.exp(alpha)
        w /= np.sum(w) # normaliza
    
    if enable_print==True:
        hist_data.append(go.Histogram(x=w.copy(), opacity=0.75))
        layout = go.Layout(barmode='stack')
        fig = go.Figure(data=hist_data, layout=layout)
        py.iplot(fig, filename='weight-histogram')
    errors = {'e_in_w_stump': w_error_stump, 'e_in_w_ens': w_error_ens}
    return ensemble, errors

# <span style='color:crimson'>5.</span> Atributos e target

Separa os conjuntos (completos) $X$ e $y$.

In [10]:
X, y = dataset[:,:-1], dataset[:,-1]

# <span style='color:crimson'>6.</span> Conjuntos de treinamento/validacao/teste

A divisão dos dados foi feita da seguinte forma:

- Há **3 conjuntos de teste**, de modo que haja mais segurança nos resultados obtidos.
- Para cada conjunto de teste, há **5 folds** para validação cruzada.
    - Cada **fold** consiste em um **conjunto de treinamento** e **conjunto de validação**.
    
A seleção foi feita de modo **estratificado**; ou seja, em todos os conjuntos de treinamento/teste/validação é mantida a proporção de amostras positivas (`'p'`) e negativas (`'n'`), sob a premissa de que o dataset é representativo da situação real.

In [11]:
def split_data(X, y, n_tests):
    n = len(y)                   # numero de amostras
    p_prop = sum(y=='p') / n     # proporcao de positivos
    n_prop = sum(y=='n') / n     # proporcao de negativos
    p_idx = np.where(y=='p')[0]  # indices dos positivos
    n_idx = np.where(y=='n')[0]  # indices dos negativos

    # Numero de amostras de cada tipo que devem haver nos testes
    nt = int(round(0.15*n))
    npt = int(round(nt * p_prop))
    nnt = nt - npt

    # 10 testes
    splits = []
    prev_test_idx = []
    for j,i in enumerate([0,3,7]):
        np.random.seed(i)

        # Seleciona indices de teste e train/val
        p_idx_test = np.random.choice(p_idx, npt, replace=False)
        n_idx_test = np.random.choice(n_idx, nnt, replace=False)
        p_idx_train_val = np.setdiff1d(p_idx, p_idx_test)
        n_idx_train_val = np.setdiff1d(n_idx, n_idx_test)

        # Junta positivos e negativos
        test_idx = np.concatenate((p_idx_test, n_idx_test))
        train_val_idx = np.setdiff1d(np.arange(n), test_idx)

        # Validacao: treino/val != teste
        assert len(np.intersect1d(train_val_idx, test_idx))==0
        
        # Seleciona amostras
        X_train_val, X_test = X[train_val_idx], X[test_idx]
        y_train_val, y_test = y[train_val_idx], y[test_idx]

        # Imprime
        print('Teste #%d' % (j+1))
        print('Train_val["p" "n"]:[%3d %3d]=%3d'%(sum(y_train_val=='p'),sum(y_train_val=='n'),len(y_train_val)))
        print('Test["p" "n"]:     [%3d %3d]=%3d'%(sum(y_test=='p'),     sum(y_test=='n'),     len(y_test)))

        # Separa em 5 folds aleatorios para validacao cruzada
        np.random.shuffle(p_idx_train_val)
        p_idx_val = np.array_split(p_idx_train_val, 5)
        np.random.shuffle(n_idx_train_val)
        n_idx_val = np.array_split(n_idx_train_val, 5)
        val = [np.concatenate((p_idx_val[k],n_idx_val[k])) for k in range(5)]
        train = [np.setdiff1d(train_val_idx, val[k]) for k in range(5)]

        # Salva
        splits.append({'test': test_idx, 'val': val, 'train': train})

        # Valida e imprime detalhes 
        prev_val_idx=[]
        for k in range(5):
            train_idx = train[k]
            val_idx = val[k]

            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            # Validacao: treino!=val e conjuntos de validacao sao mutuamente exclusivos
            assert len(np.intersect1d(train_idx, val_idx))==0
            assert len(np.intersect1d(train_idx, test_idx))==0
            assert len(np.intersect1d(val_idx, test_idx))==0
            for pvi in prev_val_idx:
                assert len(np.intersect1d(pvi, val_idx))==0
            prev_val_idx.append(val_idx)

            # Imprime
            print('\tFold #%d' % k)
            print('\tTrain["p" "n"]: [%3d %3d] = %3d' % (sum(y_train=='p'), sum(y_train=='n'), len(y_train)))
            print('\tVal["p" "n"]:   [%3d %3d] = %3d' % (sum(y_val=='p'),   sum(y_val=='n'),   len(y_val)))

    return splits

In [12]:
# Cria e imprime os splits de cada teste/(5-fold treino/validacao)
# Destaca a estratificacao
splits = split_data(X, y, n_tests=3)

Teste #1
Train_val["p" "n"]:[532 282]=814
Test["p" "n"]:     [ 94  50]=144
	Fold #0
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #1
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #2
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #3
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #4
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
Teste #2
Train_val["p" "n"]:[532 282]=814
Test["p" "n"]:     [ 94  50]=144
	Fold #0
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #1
	Train["p" "n"]: [425 225] = 650
	Val["p" "n"]:   [107  57] = 164
	Fold #2
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #3
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
	Fold #4
	Train["p" "n"]: [426 226] = 652
	Val["p" "n"]:   [106  56] = 162
Teste #3
Train_val["p" "n"]:[532 282]=814
Test["p" "n"]:     [ 94  50]=144
	Fold #0
	Train["p" "n"]:

# <span style='color:crimson'>7.</span> Experimentos

- Nesta seção estão listados os experimentos realizados e as conclusões de cada um.

## <span style='color:orange'>7.1.</span> Comportamento geral do erro *in-sample* do AdaBoost

Utilizando **um** conjunto de treinamento (a saber: primeiro fold do primeiro teste), é avaliado:
1. Qual é o erro empírico ($E_{in}$) considerando os preditores individualmente (de cada iteração).
- Qual é o erro empírico ponderado ($E_{in}^{weighted}$) considerando os preditores individualmente (de cada iteração).
- Qual é a evolução do erro empírico considerando todo o ensemble (todos os preditores) construído até aquele momento.
- Qual é a evolução do erro empírico ponderado consierando todo o ensemble (todos os preditores) construído até aquele momento.

### <span style='color:darkmagenta'>7.1.0.</span> Funções auxiliares

- Plot
- Cálculo do erro sequencial (à medida que novos preditores são incluídos) do ensemble

In [13]:
def print_e_in(errors, dtick=1):
    e_ws = errors['e_in_w_stump']
    e_we = errors['e_in_w_ens']
    e_s = errors['e_in_stump']
    e_e = errors['e_in_ens']
    layout = go.Layout(title='Erros',
                       legend=dict(orientation="v"),
                       xaxis=dict(title='#stumps', dtick=dtick),
                       yaxis=dict(title='Taxa de erro', showticklabels= True)
                      )
    e_ws_trace = go.Scatter(x=np.arange(1,len(e_ws)+1), 
                            y=e_ws, 
                            name='Erro de treinamento ponderado dos stumps'
                           )
    e_we_trace = go.Scatter(x=np.arange(1,len(e_we)+1), 
                            y=e_we, 
                            name='Erro de treinamento ponderado do ensemble'
                           )
    e_s_trace  = go.Scatter(x=np.arange(1,len(e_s)+1), 
                            y=e_s, 
                            name='Erro de treinamento dos stumps'
                           )
    e_e_trace  = go.Scatter(x=np.arange(1,len(e_e)+1), 
                            y=e_e, 
                            name='Erro de treinamento do ensemble'
                           )
    fig = go.Figure(data=[e_ws_trace, e_we_trace, e_s_trace, e_e_trace], layout=layout)
    py.iplot(fig)

In [14]:
# Calcula erros empiricos
def get_e_in(ensemble, X, y):
    n_stumps = len(ensemble)
    errors = {}
    errors['e_in_stump'] = np.empty(n_stumps)
    errors['e_in_ens'] = np.empty(n_stumps)
    for i in range(len(ensemble)):
        y_pred = evaluate_node(ensemble[i][1], X)
        errors['e_in_stump'][i] = 1 - hit_rate(y_real=y, y_pred=y_pred)
        y_pred = evaluate_ensemble(ensemble[:i+1], X)
        errors['e_in_ens'][i] = 1 - hit_rate(y_real=y, y_pred=y_pred)
    return errors

### <span style='color:darkmagenta'>7.1.1.</span> 10 preditores

- Imprime o criterio de cada stump
- Imprime histograma com distribuicoes de pesos das amostras a cada iteracao
- Imprime erros explicados acima

In [15]:
# Escolhe um conjunto de treinamento qualquer
i_test = 0
i_fold = 0
train_idx = splits[i_test]['train'][i_fold]
X_train = X[train_idx]
y_train = y[train_idx]

# Configuracoes
n_stumps = 10
learning_rate = 1

# Executa AdaBoost
ensemble, e_in_w = fit_adaboost(X_train, y_train, n_stumps,learning_rate, enable_print=True, get_errors=True)

# Calcula erros nao-ponderados
e_in = get_e_in(ensemble, X_train, y_train)

# Une erros e imprime evolucao
errors = {**e_in_w, **e_in}
print_e_in(errors,dtick=5)

Stump #0 - Classifique como "n" se x[4]==o; "p", caso contrario
Stump #1 - Classifique como "n" se x[0]==o; "p", caso contrario
Stump #2 - Classifique como "n" se x[6]==o; "p", caso contrario
Stump #3 - Classifique como "n" se x[2]==o; "p", caso contrario
Stump #4 - Classifique como "n" se x[8]==o; "p", caso contrario
Stump #5 - Classifique como "p" se x[4]==x; "n", caso contrario
Stump #6 - Classifique como "p" se x[6]==x; "n", caso contrario
Stump #7 - Classifique como "p" se x[0]==x; "n", caso contrario
Stump #8 - Classifique como "n" se x[7]==o; "p", caso contrario
Stump #9 - Classifique como "p" se x[2]==x; "n", caso contrario


### <span style='color:darkmagenta'>7.1.2.</span> 500 preditores

Demonstra evolução do erro *in-sample* para muitos preditores

**IMPORTANTE** O bloco a seguir demora alguns minutos para executar (aprox. **5 minutos**). 

In [16]:
i_test = 0
i_fold = 0

train_idx = splits[i_test]['train'][i_fold] # conjunto de treinamento qualquer
X_train = X[train_idx]
y_train = y[train_idx]

n_stumps = 500
learning_rate = 1

# Executa AdaBoost
ensemble, e_in_w = fit_adaboost(X_train, y_train, n_stumps,learning_rate, enable_print=False, get_errors=True)

# Calcula erros nao-ponderados
e_in = get_e_in(ensemble, X_train, y_train)

# Une erros e imprime evolucao
errors = {**e_in_w, **e_in}
print_e_in(errors,dtick=20)

### <span style='color:darkmagenta'>7.1.3. Conclusão</span>:

- O erro de treinamento do ensemble demonstra que o *AdaBoost* está funcionando, pois o erro cai à medida que novos preditores são adicionados, o que é de se esperar, pois o modelo se ajusta mais precisamente aos dados (ignorando *overfitting*).
- O erro ponderado do ensemble demonstra que há um ponto em que deixa de 'valer a pena' prosseguir com o treinamento (aprox. 200 preditores), que é onde sua curva deixa de cair e passa a crescer.
- O erro de treinamento dos stumps individuais parece uma onda pois há apénas 27 possibilidades de preditores (9 posições com 3 valores cada). Logo, há uma repetição à medida que os mesmos critérios são utilizados novamenteo.
- O erro ponderado para preditores individuais aparenta manter uma taxa quase constante de aproximadamente $0.45$. Isto demonstra que a atualização dos pesos faz com que as amostras mais difíceis de fato tenham mais peso na hora de avaliar o desempenho.

## <span style='color:orange'>7.2.</span> *Tradeoff* viés-variância

Este experimento busca avaliar como é o tradeoff de viés e variância em função do aumento do número de preditores (a taxa de aprendizado também poderia ser utilizada, mas foi utilizado como parâmetro apenas o número de preditores).

Para isso, será feito o que está descrito abaixo.

- Para cada conjunto de teste, avalie:
    - Para o ensemble construído até a iteração $i$, avalie:
        - Média dos erros empíricos $E_{in}$
        - Erro da validação cruzada $E_{val}$
        - Erro do conjunto de teste $E_{out}$

### <span style='color:darkmagenta'>7.2.0.</span> Funções auxiliares

- Retorna os erros de treinamento/validação/teste à medida em que novos preditores são incluídos no ensemble.
- Plot

In [17]:
def get_ensemble_errors(ensemble, X, y):
    n = X.shape[0]
    y_pred = np.full(n, fill_value='n')
    votes = [{'n':0, 'p':0} for i in range(n)]
    errors = np.empty(n_stumps)
    for i_stump, (alpha,stump) in enumerate(ensemble):
        stump_preds = evaluate_node(stump, X)
        for i_x, pred in enumerate(stump_preds): 
            votes[i_x][pred] += alpha
            y_pred[i_x] = 'p' if votes[i_x]['p'] >= votes[i_x]['n'] else 'n'
        errors[i_stump] = 1 - hit_rate(y_real=y, y_pred=y_pred)
    return errors

In [18]:
# Calcula erros empiricos
def print_errors(E_in, E_val, E_out, E_in_w=None, dtick=1):
    layout = go.Layout(title='Erros',
                       legend=dict(orientation="v"),
                       xaxis=dict(title='#stumps', dtick=dtick),
                       yaxis=dict(title='Taxa de erro', showticklabels= True)
                      )
    E_in_trace = go.Scatter(x=np.arange(1,len(E_in)+1), 
                            y=E_in, 
                            name='Erro de treinamento'
                           )
    E_val_trace = go.Scatter(x=np.arange(1,len(E_val)+1), 
                            y=E_val, 
                            name='Erro de validacao'
                           )
    E_out_trace  = go.Scatter(x=np.arange(1,len(E_out)+1), 
                            y=E_out, 
                            name='Erro de teste'
                           )
    trace_arr = [E_in_trace, E_val_trace, E_out_trace]
    
    if E_in_w is not None:
        E_in_w_trace  = go.Scatter(x=np.arange(1,len(E_in_w)+1), 
                                y=E_in_w, 
                                name='Erro ponderado de treinamento'
                               )
        trace_arr.append(E_in_w_trace)
        
    fig = go.Figure(data=trace_arr,layout=layout)
    py.iplot(fig)

### <span style='color:darkmagenta'>7.2.1.</span> Parâmetro

Será variado o número de preditores no intervalo $[1,500]$ e avaliados os erros obtidos por cada um.

In [19]:
n_tests = len(splits)
n_stumps = 500

E_in_list = []
E_val_list = []
E_out_list = []

for i_test in range(n_tests):
    print('\nTeste #%d' % (i_test+1))
    test_idx = splits[i_test]['test']
    X_test, y_test = X[test_idx], y[test_idx]
    
    E_in_w = np.zeros(n_stumps)
    E_in = np.zeros(n_stumps)
    E_val = np.zeros(n_stumps)
    E_out = np.zeros(n_stumps)
    
    # Calcula as medias de E_in e E_val
    print('Fold #', end='')
    for i_fold in range(5):
        print('%d' % i_fold, end='/')
        train_idx, val_idx = splits[i_test]['train'][i_fold], splits[i_test]['val'][i_fold]
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        
        # Executa AdaBoost
        ensemble,_ = fit_adaboost(X_train,y_train,n_stumps,learning_rate,enable_print=False,get_errors=False)

        # Calcula erros nao-ponderados
        E_in += get_ensemble_errors(ensemble, X_train, y_train)
        E_val += get_ensemble_errors(ensemble, X_val, y_val)
        
    # Calcula E_out
    E_out = get_ensemble_errors(ensemble, X_test, y_test)
    E_in /= 5
    E_val /= 5
    
    E_in_list.append(E_in)
    E_val_list.append(E_val)
    E_out_list.append(E_out)


Teste #1
Fold #0/1/2/3/4/
Teste #2
Fold #0/1/2/3/4/
Teste #3
Fold #0/1/2/3/4/

### <span style='color:darkmagenta'>7.2.2.</span> Imprime resultados

Para cada teste, imprime:
- Seleção de modelo:
    - Quantos stumps seriam selecionados por validação cruzada, i.e., qual modelo seria escolhido.
    - Qual o valor mínimo para o conjunto de validação (correspondente ao modelo selecionado).
    - Qual o erro de treinamento este modelo obtém.
    - Qual o erro de teste este modelo obtém.
- Gráfico da evolução dos erros:
    - Evolução do erro de treinamento - $E_{in}$.
    - Evolução do erro de treinamento - $E_{val}$.
    - Evolução do erro de treinamento - $E_{out}$.

In [20]:
n_tests = len(splits)
for i in range(n_tests):
    # Selecao por validacao cruzada:
    n_stumps_opt = np.argmin(E_val_list[i])
    print('\nTeste #%d' % (i+1))
    print('Numero de stumps selecionado por validacao cruzada 5-fold: %d' % (n_stumps_opt+1))
    print('Erro minimo de validacao cruzada: %f' % E_val_list[i][n_stumps_opt])
    print('Erro de treinamento / risco empirico / E_in: %f' % E_in_list[i][n_stumps_opt])
    print('Erro de teste / risco esperado / E_out: %f' % E_out_list[i][n_stumps_opt])
    print_errors(E_in=E_in_list[i], E_val=E_val_list[i], E_out=E_out_list[i], dtick=20)


Teste #1
Numero de stumps selecionado por validacao cruzada 5-fold: 323
Erro minimo de validacao cruzada: 0.018458
Erro de treinamento / risco empirico / E_in: 0.019045
Erro de teste / risco esperado / E_out: 0.013889



Teste #2
Numero de stumps selecionado por validacao cruzada 5-fold: 303
Erro minimo de validacao cruzada: 0.015959
Erro de treinamento / risco empirico / E_in: 0.016585
Erro de teste / risco esperado / E_out: 0.020833



Teste #3
Numero de stumps selecionado por validacao cruzada 5-fold: 254
Erro minimo de validacao cruzada: 0.013520
Erro de treinamento / risco empirico / E_in: 0.014128
Erro de teste / risco esperado / E_out: 0.034722


### <span style='color:darkmagenta'>7.2.3. Conclusão</span>:

A conclusão mais relevante é a de que **não há overfitting neste problema**. Este fato pode ser verificado ao observar que **todas as curvas de erro caem** à medida que a flexibilidade (número de preditores) do modelo aumenta. Em caso de *overfitting*, observaríamos as curvas de validação/teste possuírem um formato de "v", o que não é o caso aqui.

A explicação para isto é que **todo o dataset é representativo**, pois suas classes são deterministicamente geradas a partir de regras específicas baseadas apenas em atributos existentes (as 9 posições do tabuleiro). Ou seja, não há "ruído" nos dados - ao qual um modelo muito flexível pudesse se ajustar e perder generalidade. É como uma regressão linear em que todas as amostras estão exatamente sobre a mesma linhas. Em outras palavras, o aumento do número de preditores só ajuda o modelo a aprender melhor as "regras" do jogo da velha.

## <span style='color:orange'>7.3.</span> Learning Curves

Um último teste realizado é o de plotar as curvas de aprendizado de modelos com diferentes hiperparâmetros (número de preditores). Em geral elas são muito úteis para identificar viés-variância, mas não existe este *tradeoff* em nosso problema (experimento 7.2), é esperado que só seja possível identificar viés para modelos mais simples.

- Para um conjunto arbitrário de teste/treinamento, é feito:
    - Para 5 modelos com flexibilidades diferentes (`n_stumps` = 10, 20, 50, 100, 300)
        - Para k = 80,90,100,... amostras de treinamento:
            - Ajusta o modelo utilizando k amostras (sempre mantendo a proporção entre as classes).
            - Avalia $E_{in}$ e $E_{out}$ para esta combinação (modelo/número de amostras).
            

### <span style='color:darkmagenta'>7.3.0.</span> Função auxiliar

- Plot

In [21]:
# Calcula erros empiricos
def print_learning_curve(lc, samples_vec):
    for n_pred, curves in lc.items():
        layout = go.Layout(title='Erros com %d preditores' % n_pred,
                           legend=dict(orientation="v"),
                           xaxis=dict(title='#amostras'),
                           yaxis=dict(title='Taxa de erro', showticklabels= True)
                          )
        E_in_trace = go.Scatter(x=samples_vec, 
                                y=curves[0], 
                                name='Erro de treinamento'
                               )
        E_out_trace  = go.Scatter(x=samples_vec, 
                                y=curves[1], 
                                name='Erro de teste'
                               )
        fig = go.Figure(data=[E_in_trace, E_out_trace],layout=layout)
        py.iplot(fig)

### <span style='color:darkmagenta'>7.3.1.</span> Executa experimento

In [22]:
# Conjunto arbitrario de treinamento/teste
i_test = 0
i_train = 0

test_idx = splits[i_test]['test']
train_idx= splits[i_test]['train'][i_train]
val_idx = splits[i_test]['val'][i_train]

train_idx = np.concatenate([train_idx,val_idx]) # podemos unir os conjuntos de treinamento e validacao

y_lc = y[train_idx]
n = len(y_lc)                   # numero de amostras
p_prop = sum(y_lc=='p') / n     # proporcao de positivos
n_prop = sum(y_lc=='n') / n     # proporcao de negativos
p_idx = np.where(y_lc=='p')[0]  # indices dos positivos
n_idx = np.where(y_lc=='n')[0]  # indices dos negativos

In [23]:
lc = {} # learning curves
ntr = train_idx.shape[0]
samples_vec = np.arange(80,ntr,10) # numero crescente de amostras

X_test, y_test = X[test_idx], y[test_idx]
X_lc, y_lc = X[train_idx], y[train_idx]

for n_stumps in [10, 20, 50, 100, 300]:
    E_in_vec = np.empty(samples_vec.shape)
    E_out_vec = np.empty(samples_vec.shape)
    for i,n_samples in enumerate(samples_vec):
        npos = int(round(p_prop*n_samples))
        nneg = n_samples - npos
        idx = np.concatenate((p_idx[:npos], n_idx[:nneg]))
        X_train, y_train = X_lc[idx], y_lc[idx]
        
        # Executa AdaBoost
        ensemble,_ = fit_adaboost(X_train,y_train,n_stumps)
        
        # E_in e E_out
        y_in_pred = evaluate_ensemble(ensemble, X_train)
        E_in_vec[i]  = 1 - hit_rate(y_pred=y_in_pred, y_real=y_train)
        y_out_pred = evaluate_ensemble(ensemble, X_test)
        E_out_vec[i] = 1 - hit_rate(y_pred=y_out_pred, y_real=y_test)
        
    lc[n_stumps] = (E_in_vec, E_out_vec)

### <span style='color:darkmagenta'>7.3.2.</span> Imprime resultados

Imprime as curvas de aprendizado de cada hiperparâmetro (10, 20, 50, 100 e 300 preditores)

In [24]:
print_learning_curve(lc,samples_vec)

### <span style='color:darkmagenta'>7.3.3. Conclusão</span>:

- O experimento confirma que de fato não há problema de variância neste trabalho, pois as curvas de $E_{in}$ e $E_{out}$ estão sempre próximas.

- Há mais viés quão mais simples o modelo é como destacado pelos valores finais de $E_{in}$ e $E_{out}$ em cada caso:
    - 10 preditores:  ~$30\%$
    - 20 preditores:  ~$25\%$
    - 50 preditores:  ~$20\%$
    - 100 preditores: ~$10\%$
    - 300 preditores: ~$2\%$